In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import array_to_img
from utils.utils import load_local_dataset_tf, preprocess_data_per_tfmodel
from utils.constants import BASE_PATH

In [2]:
TARGET_SIZE = (224, 224)
TARGET_DATA_PATH = BASE_PATH + "data/target/"

In [3]:
train_ds = load_local_dataset_tf(
    TARGET_DATA_PATH + "mechanicalseals_fulllight/", 
    target_size=TARGET_SIZE, 
    subset="training",
    seed=42,
    batch_size=1
)
                    
test_ds = load_local_dataset_tf(
    TARGET_DATA_PATH + "mechanicalseals_fulllight/", 
    target_size=TARGET_SIZE, 
    subset="test",
    seed=42,
    batch_size=1
)

train_ds = preprocess_data_per_tfmodel(train_ds)
test_ds = preprocess_data_per_tfmodel(test_ds)

# [0, 1] intact
# [1, 0] damaged

Found 181 files belonging to 2 classes.
Using 145 files for training.
Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Found 181 files belonging to 2 classes.
Using 36 files for validation.


2023-03-18 18:01:09.423112: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-18 18:01:09.423267: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:

# Collect images and labels of the desired class
train_images_defect = []
train_labels_defect = []
train_images_intact = []
train_labels_intact = []

i = 0
for image, label in train_ds.unbatch():
    if label.numpy()[0] == 0: # intact
        train_images_intact.append(image.numpy())
        train_labels_intact.append(label.numpy())
    elif label.numpy()[0] == 1: # defect
        train_images_defect.append(image.numpy())
        train_labels_defect.append(label.numpy())
    
    i += 1
    # if i == 3:
    #     break
    
print("defect - train: ", len(train_images_defect))
print("instact - train: ", len(train_images_intact))
    
    
# Collect images and labels of the desired class
# test_images_defect = []
# test_labels_defect = []
# test_images_intact = []
# test_labels_intact = []

# i = 0
# for image, label in test_ds.unbatch():
#     if label.numpy()[0] == 0: # intact
#         test_images_intact.append(image.numpy())
#         test_labels_intact.append(label.numpy())
#     elif label.numpy()[0] == 1: # defect
#         test_images_defect.append(image.numpy())
#         test_labels_defect.append(label.numpy())
    
#     i += 1
#     # if i == 3:
#     #     break
    

# print("defect - test: ", len(test_images_defect))
# print("intact - test: ", len(test_images_intact))

# train_images_defect = np.array(train_images_defect)
# train_labels_defect = np.array(train_labels_defect)
# train_defect = tf.data.Dataset.from_tensor_slices((train_images_defect, train_labels_defect))

# train_images_intact = np.array(train_images_intact)
# train_labels_intact = np.array(train_labels_intact)
# train_intact = tf.data.Dataset.from_tensor_slices((train_images_intact, train_labels_intact))



2023-03-18 18:01:09.574648: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


defect - train:  76
instact - train:  69


In [5]:

# k = 1 

# k_shot_train_preprocessed_defect = train_defect.take(k)
# k_shot_train_preprocessed_intact = train_intact.take(k)

In [7]:
from keras.applications.resnet import ResNet101

no_top = ResNet101(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

with_top = ResNet101(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

179648224/179648224 [==============================] - 87s 0us/step


In [8]:
print("no_top")
no_top.summary()

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                       

In [9]:
print("with_top")
with_top.summary()

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          